In [ ]:
import pandas as pd

cpi_price = pd.read_excel("GemDataEXTR/CPI Price, nominal, not seas. adj..xlsx", sheet_name="monthly")
core_price = pd.read_excel("GemDataEXTR/Core CPI, not seas. adj..xlsx", sheet_name="monthly")
exchange_rate = pd.read_excel("GemDataEXTR/Exchange rate, new LCU per USD extended backward, period average.xlsx", sheet_name="monthly")
exports_merch = pd.read_excel("GemDataEXTR/Exports Merchandise, Customs, current US$, millions, not seas. adj..xlsx", sheet_name="monthly")
gdp = pd.read_excel("GemDataEXTR/GDP at market prices, current LCU, millions, seas. adj..xlsx", sheet_name="quarterly")
imports_merch = pd.read_excel("GemDataEXTR/Imports Merchandise, Customs, Price, US$, not seas. adj..xlsx", sheet_name="monthly")
industrial_prod = pd.read_excel("GemDataEXTR/Industrial Production, constant 2010 US$, not seas. adj..xlsx", sheet_name="monthly")
effective_exchange_rate = pd.read_excel("GemDataEXTR/Nominal Effective Exchange Rate.xlsx", sheet_name="monthly")
retail_sales_vol = pd.read_excel("GemDataEXTR/Retail Sales Volume Index, seas. adj..xlsx", sheet_name="monthly")
stock_market = pd.read_excel("GemDataEXTR/Stock Markets, LCU.xlsx", sheet_name="monthly")
tot = pd.read_excel("GemDataEXTR/Terms of Trade.xlsx", sheet_name="monthly")
unemployment_rate = pd.read_excel("GemDataEXTR/Terms of Trade.xlsx", sheet_name="monthly")
Japan_CPI = pd.read.excel("Consumer Price Index All Items Total for Japan.xlsx", sheet_name="monthly")
